In [ ]:
import datetime as dt
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler # use this to stream in the terminal
import httpx

In [3]:
# tools.py (original file)

class Toolbox:
    '''
    A class representing possible tools to be utilized by agent.
    '''
    def __init__(self): # for class initialization
        print("Toolbox instance has been created")

    #====================================================================================================

    # utilizes datetime package referenced above
    def get_current_datetime(self, input_param=""):
        """Get current date and time in EST standard time"""
        print("get_date_time_tool invoked\n")
        est = dt.datetime.now() - dt.timedelta(hours=5)  # EST is UTC-5 (or -4 for EDT)
        return f"Current EST Date/Time: {est.strftime('%Y-%m-%d %H:%M:%S')}"

In [2]:
# agent.py (original file)
def create_tools():
    '''
    Creates tool instances for agent

    Parameters:
    None
    

    Returns:
    list: List of tools to be used by agent
    '''

    # Initialize Toolbox object.
    print("Creating toolbox instance")
    toolbox = Toolbox() # creates Toolbox object so we can access our tools in toolbox. doesn't have to be this way but I figured it made sense like this

    # create tool instance here to return as a list so our AgentExecutor can use/understand it
    date_time_tool = Tool(
    name="get_current_datetime",
    description="Use this tool to get the date and time", # prompt for the agent to understand what the tool does
    func=toolbox.get_current_datetime # gets the tool/function you made in toolbox class
    )

    # add more tools here depending on what you need to do (query from seek and draw insights, create csv/pdf output tool, etc.)

    tools = [date_time_tool] # return tools as a list.
    return tools

In [6]:
# agent.py (original file)
def create_chatbot(tools: list[Tool]) -> AgentExecutor:
    '''
    Creates chatbot/agent that is integrated with LLM. This is essentially the core of the chatbot

    Parameters:
    tools (list[Tool]): List of Tools instances

    Returns:
    AgentExecutor: Agent object that will be called by main to execute agent. This is essentially the core of the chatbot
    '''
    
    # you need this to bypass the annoying firewall
    httpx_client = httpx.Client(http2=False, verify=False)
    
    # initialize LLM object
    llm = AzureChatOpenAI(
        azure_endpoint='https://air-api.regeneron.com/v1.0/model',
        api_version='2024-08-01-preview',
        http_client=httpx_client,
        api_key="AIR-API",
        azure_deployment="gpt-4o", # choose the model you want to use
        model_name="gpt-4", 
        default_headers={
            'authorization-token': '' # api here
        },
        temperature=0.1
    )
    
    print("Creating agent...")
    
    # Initialize memory for chat history (every time you run this function again, it resets)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True) 
    
    # Create prompt template, basically a more in-depth messages template [("system", ""), ("human", "")]
    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a helpful AI assistant"), # system prompt so the LLM knows what its role is
        MessagesPlaceholder(variable_name="chat_history"), #specifies what key to access ConversationBufferMemory
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad") # used for agent's working space (mostly as placeholder)
    ])
    
    # Create agent. we need to do this because AgentExecutor() requires us to pass in the agent object.
    agent = create_openai_functions_agent(
        llm=llm,
        tools=tools,
        prompt=prompt_template
    )
    
    # Create agent executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=False
    )
    
    print("\nAI Documentation Assistant is ready.")
    return agent_executor


In [7]:
# running everything
tools = create_tools()
agent_executor = create_chatbot(tools)
user_input = "what is the time right now"

# agent_executor is similar to llm here https://python.langchain.com/docs/integrations/chat/azure_chat_openai/,
# except it is an agent with tools instead of just the basic LLM
response = agent_executor.invoke({"input": user_input})
print(response["output"])

Creating toolbox instance
Toolbox instance has been created
Creating agent...

AI Documentation Assistant is ready.


C:\Users\daniel.canhedo\AppData\Local\Temp\1\ipykernel_29800\2968526665.py:32: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


get_date_time_tool invoked

The current date and time is July 8, 2025, at 9:19 AM (EST).
